In [ ]:
!pip install kombu

import threading
import time
import logging
import uuid
import datetime
import sys

from kombu import Connection, Exchange, Queue, Producer, Consumer
from kombu.mixins import ConsumerMixin

logging.basicConfig(
    level=logging.INFO,
    format='%(message)s',
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)
logger = logging.getLogger(__name__)

BROKER_URL = "memory://localhost/"

In [ ]:
media_exchange = Exchange('media_exchange', type='topic', durable=True)

task_queues = [
    Queue('video_queue', media_exchange, routing_key='video.#'),
    Queue('audit_queue', media_exchange, routing_key='#'),
]

In [ ]:
class Worker(ConsumerMixin):
    def __init__(self, connection, queues):
        self.connection = connection
        self.queues = queues
        self.should_stop = False

    def get_consumers(self, Consumer, channel):
        return [
            Consumer(queues=self.queues,
                     callbacks=[self.on_message],
                     accept=['json'],
                     prefetch_count=1)
        ]

    def on_message(self, body, message):
        routing_key = message.delivery_info['routing_key']
        payload_id = body.get('id', 'unknown')

        logger.info(f"\n⚡ RECEIVED MSG via key: [{routing_key}]")
        logger.info(f"   Payload ID: {payload_id}")

        try:
            if 'video' in routing_key:
                self.process_video(body)
            elif 'audit' in routing_key:
                logger.info("   🔍 [Audit] Logging event...")

            message.ack()
            logger.info(f"   ✅ ACKNOWLEDGED")

        except Exception as e:
            logger.error(f"   ❌ ERROR: {e}")

    def process_video(self, body):
        logger.info("   ⚙️  [Processor] Transcoding video (Simulating work...)")
        time.sleep(0.5)

In [ ]:
def publish_messages(connection):
    producer = Producer(connection)

    tasks = [
        ('video.upload', {'file': 'movie.mp4'}),
        ('user.login', {'user': 'admin'}),
    ]

    logger.info("\n🚀 PRODUCER: Starting to publish messages...")

    for r_key, data in tasks:
        data['id'] = str(uuid.uuid4())[:8]

        logger.info(f"📤 SENDING: {r_key} -> {data}")

        producer.publish(
            data,
            exchange=media_exchange,
            routing_key=r_key,
            serializer='json'
        )
        time.sleep(1.5)

    logger.info("🏁 PRODUCER: Done.")

In [10]:
def run_example():
    with Connection(BROKER_URL) as conn:
        worker = Worker(conn, task_queues)
        worker_thread = threading.Thread(target=worker.run)
        worker_thread.daemon = True
        worker_thread.start()

        logger.info("✅ SYSTEM: Worker thread started.")
        time.sleep(1)

        try:
            publish_messages(conn)
            time.sleep(2)
        except KeyboardInterrupt:
            pass
        finally:
            worker.should_stop = True
            logger.info("\n👋 SYSTEM: Execution complete.")

if __name__ == "__main__":
    run_example()

Connected to memory://localhost//
✅ SYSTEM: Worker thread started.

🚀 PRODUCER: Starting to publish messages...
📤 SENDING: video.upload -> {'file': 'movie.mp4', 'id': 'b4da5c90'}

⚡ RECEIVED MSG via key: [video.upload]
   Payload ID: b4da5c90
   ⚙️  [Processor] Transcoding video (Simulating work...)
📤 SENDING: user.login -> {'user': 'admin', 'id': '48968142'}
   ✅ ACKNOWLEDGED

⚡ RECEIVED MSG via key: [video.upload]
   Payload ID: b4da5c90
   ⚙️  [Processor] Transcoding video (Simulating work...)
   ✅ ACKNOWLEDGED

⚡ RECEIVED MSG via key: [user.login]
   Payload ID: 48968142
   ✅ ACKNOWLEDGED
🏁 PRODUCER: Done.

👋 SYSTEM: Execution complete.
